# Predicción de crímenes
### Trabajo Fin de Máster para el Máster Universitario en Ciencia de Datos
### Universitat Oberta de Catalunya
### Realizado por Álvaro Pavón Díaz

## Parte 2: Modelado y evaluación de los datos
Durante este Jupyter Notebook nos dedicaremos a realizar el entrenamiento de los conjunto de datos sobre diferentes métodos.

Para ello lo primero se va a proceder a importar las diferentes clases que se necesitarán a lo largo de este notebook y la importación de los archivos que se va a utilizar. Además de mostrar que los proceso realizados a través de TensorFlow van a ser acelerados mediante GPU (GeForce RTX 2070 SUPER)

In [1]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time_function
import tensorflow as tf
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1" 

print(device_lib.list_local_devices())
print(tf.version)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14663795888489606148
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 13305875057602867155
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5175569564501812678
physical_device_desc: "device: XLA_GPU device"
]
<module 'tensorflow._api.v2.version' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>


In [2]:
crimes = pd.read_csv ('crimes_clean.csv', sep='""', delimiter=',', engine='python')
crimes.head(5)

,OFFENSE_CODE_GROUP,DISTRICT,SHOOTING,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,IS_NIGHT,DAY_OF_MONTH
0,Disorderly Conduct,E18,0.0,2018,10,Wednesday,20,Part Two,ARLINGTON ST,42.262608,-71.121186,1,3
1,Property Lost,D14,0.0,2018,8,Thursday,20,Part Three,ALLSTON ST,42.352111,-71.135311,1,30
2,Other,B2,0.0,2018,10,Wednesday,19,Part Two,DEVON ST,42.308126,-71.076930,1,3
3,Aggravated Assault,A1,0.0,2018,10,Wednesday,20,Part One,CAMBRIDGE ST,42.359454,-71.059648,1,3
4,Aircraft,A7,0.0,2018,10,Wednesday,20,Part Three,PRESCOTT ST,42.375258,-71.024663,1,3


In [3]:
crimes_time = pd.read_csv ('crimes_clean_time.csv', sep='""', delimiter=',', engine='python')
crimes_time.head(5)

,OFFENSE_CODE_GROUP,DISTRICT,SHOOTING,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,IS_NIGHT,DAY_OF_MONTH,TIME_X,TIME_Y,Lat_M,Long_M
0,Disorderly Conduct,E18,0.0,2018,10,Wednesday,20,Part Two,ARLINGTON ST,42.262608,-71.121186,1,3,0.518198,0.000331,0.185665,0.267386
1,Property Lost,D14,0.0,2018,8,Thursday,20,Part Three,ALLSTON ST,42.352111,-71.135311,1,30,0.239106,0.073462,0.736023,0.201687
2,Other,B2,0.0,2018,10,Wednesday,19,Part Two,DEVON ST,42.308126,-71.076930,1,3,0.517840,0.000318,0.465558,0.473233
3,Aggravated Assault,A1,0.0,2018,10,Wednesday,20,Part One,CAMBRIDGE ST,42.359454,-71.059648,1,3,0.518198,0.000331,0.781170,0.553614
4,Aircraft,A7,0.0,2018,10,Wednesday,20,Part Three,PRESCOTT ST,42.375258,-71.024663,1,3,0.518198,0.000331,0.878350,0.716335


In [4]:
crimes_time=crimes_time[['OFFENSE_CODE_GROUP', 'SHOOTING','UCR_PART','YEAR','MONTH','DAY_OF_MONTH','DAY_OF_WEEK','HOUR','IS_NIGHT','Lat', 'Lat_M', 'Long','Long_M', 'TIME_X', 'TIME_Y', 'DISTRICT']]
crimes_time.head(5)

,OFFENSE_CODE_GROUP,SHOOTING,UCR_PART,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,HOUR,IS_NIGHT,Lat,Lat_M,Long,Long_M,TIME_X,TIME_Y,DISTRICT
0,Disorderly Conduct,0.0,Part Two,2018,10,3,Wednesday,20,1,42.262608,0.185665,-71.121186,0.267386,0.518198,0.000331,E18
1,Property Lost,0.0,Part Three,2018,8,30,Thursday,20,1,42.352111,0.736023,-71.135311,0.201687,0.239106,0.073462,D14
2,Other,0.0,Part Two,2018,10,3,Wednesday,19,1,42.308126,0.465558,-71.076930,0.473233,0.517840,0.000318,B2
3,Aggravated Assault,0.0,Part One,2018,10,3,Wednesday,20,1,42.359454,0.781170,-71.059648,0.553614,0.518198,0.000331,A1
4,Aircraft,0.0,Part Three,2018,10,3,Wednesday,20,1,42.375258,0.878350,-71.024663,0.716335,0.518198,0.000331,A7


In [5]:
crimes_times = crimes_time.sort_values(['YEAR', 'MONTH', 'DAY_OF_MONTH', 'HOUR'], ascending=[True, True, True, True])
crimes_times.head(5)

,OFFENSE_CODE_GROUP,SHOOTING,UCR_PART,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,HOUR,IS_NIGHT,Lat,Lat_M,Long,Long_M,TIME_X,TIME_Y,DISTRICT
127095,Harassment,0.0,Part Two,2015,6,15,Monday,0,1,42.291093,0.360820,-71.065945,0.524324,0.022469,0.648202,C11
241630,Property Lost,0.0,Part Three,2015,6,15,Monday,0,1,42.283634,0.314958,-71.082813,0.445868,0.022469,0.648202,B3
288821,Fraud,0.0,Part Two,2015,6,15,Monday,0,1,42.360205,0.785788,-71.056208,0.569614,0.022469,0.648202,A1
295842,Other,0.0,Part Two,2015,6,15,Monday,0,1,42.293606,0.376273,-71.071887,0.496690,0.022469,0.648202,C11
300018,Confidence Games,0.0,Part Two,2015,6,15,Monday,0,1,42.300217,0.416923,-71.080979,0.454397,0.022469,0.648202,B3


In [6]:
crimes_time_one = crimes_time
le_dw = LabelEncoder()
dw_labels = le_dw.fit_transform(crimes_time_one['DAY_OF_WEEK'])
crimes_time_one['DAY_OF_WEEK_label'] = dw_labels

one_dw = OneHotEncoder()
days_of_week = one_dw.fit_transform(crimes_time_one[['DAY_OF_WEEK_label']]).toarray()
days_of_week_labels = list(le_dw.classes_)
days_of_week_df = pd.DataFrame(days_of_week, columns=days_of_week_labels)



In [7]:
le_ocg = LabelEncoder()
ocg_labels = le_ocg.fit_transform(crimes_time_one['OFFENSE_CODE_GROUP'])
crimes_time_one['OFFENSE_CODE_label'] = ocg_labels

one_ocg = OneHotEncoder()
ocg = one_ocg.fit_transform(crimes_time_one[['OFFENSE_CODE_label']]).toarray()
ocg_labels = list(le_ocg.classes_)
ocg_df = pd.DataFrame(ocg, columns=ocg_labels)

In [8]:
#le_year = LabelEncoder()
#year_labels = le_year.fit_transform(crimes_time_one['YEAR'])
#crimes_time_one['year_label'] = year_labels

#one_year = OneHotEncoder()
#year = one_year.fit_transform(crimes_time_one[['year_label']]).toarray()
#year_labels = list(le_year.classes_)
#year_df = pd.DataFrame(year, columns=year_labels)

In [9]:
le_d = LabelEncoder()
d_labels = le_d.fit_transform(crimes_time_one['DISTRICT'])
crimes_time_one['DISTRICT_labels'] = d_labels

In [10]:
le_ucr = LabelEncoder()
ucr_labels = le_ucr.fit_transform(crimes_time_one['UCR_PART'])
crimes_time_one['UCR_PART_label'] = ucr_labels

one_ucr = OneHotEncoder()
ucr = one_ucr.fit_transform(crimes_time_one[['UCR_PART_label']]).toarray()
ucr_labels = list(le_ucr.classes_)
ucr_df = pd.DataFrame(ucr, columns=ucr_labels)

In [11]:
crimes_time_one = pd.concat([ocg_df,ucr_df,days_of_week_df, crimes_time_one], axis=1)

crimes_time_one.drop('DAY_OF_WEEK_label', 1, inplace= True)
crimes_time_one.drop('OFFENSE_CODE_label', 1, inplace= True)
crimes_time_one.drop('DAY_OF_WEEK', 1, inplace= True)
crimes_time_one.drop('OFFENSE_CODE_GROUP', 1, inplace= True)
#crimes_time_one.drop('year_label', 1, inplace= True)
crimes_time_one.drop('DISTRICT', 1, inplace= True)
crimes_time_one.drop('Lat', 1, inplace= True)
crimes_time_one.drop('Long', 1, inplace= True)
crimes_time_one.drop('Lat_M', 1, inplace= True)
crimes_time_one.drop('Long_M', 1, inplace= True)
crimes_time_one.drop('UCR_PART', 1, inplace= True)

In [12]:
crimes_time_one = crimes_time_one.sort_values(['YEAR', 'MONTH', 'DAY_OF_MONTH', 'HOUR'], ascending=[True, True, True, True])
crimes_time_train_test = crimes_time_one.head(int(crimes_time_one.shape[0] * 0.8))
crimes_time_validation = crimes_time_one.tail(crimes_time_one.shape[0] - int(crimes_times.shape[0] * 0.8))
crimes_time_train_test.drop('YEAR', 1, inplace= True)
crimes_time_validation.drop('YEAR', 1, inplace= True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
crimes_time_train_test.head(5)

,Aggravated Assault,Aircraft,Assembly or Gathering Violations,Auto Theft,Ballistics,Biological Threat,Bomb Hoax,Commercial Burglary,Confidence Games,Counterfeiting,...,Wednesday,SHOOTING,MONTH,DAY_OF_MONTH,HOUR,IS_NIGHT,TIME_X,TIME_Y,DISTRICT_labels,UCR_PART_label
127095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6,15,0,1,0.022469,0.648202,5,2
241630,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6,15,0,1,0.022469,0.648202,4,1
288821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6,15,0,1,0.022469,0.648202,0,2
295842,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6,15,0,1,0.022469,0.648202,5,2
300018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,6,15,0,1,0.022469,0.648202,4,2


In [14]:
crimes_time_validation.head(5)

,Aggravated Assault,Aircraft,Assembly or Gathering Violations,Auto Theft,Ballistics,Biological Threat,Bomb Hoax,Commercial Burglary,Confidence Games,Counterfeiting,...,Wednesday,SHOOTING,MONTH,DAY_OF_MONTH,HOUR,IS_NIGHT,TIME_X,TIME_Y,DISTRICT_labels,UCR_PART_label
61407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,3,14,0,0.91873,0.773249,4,1
61408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,3,14,0,0.91873,0.773249,7,0
61411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,3,14,0,0.91873,0.773249,8,2
61412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,3,14,0,0.91873,0.773249,8,2
61413,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,3,14,0,0.91873,0.773249,8,2


In [15]:
crimes_time_validation.shape

(60846, 78)

In [16]:
crimes_time_train_test.shape

(243383, 78)

In [17]:
xTrain, xTest, yTrain, yTest = train_test_split(crimes_time_train_test.loc[:,'Aggravated Assault':'TIME_Y'], crimes_time_train_test.loc[:,'DISTRICT_labels'], test_size = 0.25, random_state = 0)
#xTrain, xTest, yTrain, yTest = train_test_split(crimes_time_train_test.loc[:,'OFFENSE_CODE_GROUP':'TIME_Y'], crimes_time_train_test.loc[:,'DISTRICT'], test_size = 0.25, random_state = 0)

In [18]:
#xTrain.drop([ 'TIME_X', 'TIME_Y'], 1, inplace= True)
#xTest.drop([ 'TIME_X', 'TIME_Y'], 1, inplace= True)
xTrain.drop([ 'MONTH', 'DAY_OF_MONTH', 'HOUR'], 1, inplace= True)
xTest.drop([ 'MONTH', 'DAY_OF_MONTH', 'HOUR'], 1, inplace= True)

In [19]:
xTrain.head(15)

,Aggravated Assault,Aircraft,Assembly or Gathering Violations,Auto Theft,Ballistics,Biological Threat,Bomb Hoax,Commercial Burglary,Confidence Games,Counterfeiting,...,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,SHOOTING,IS_NIGHT,TIME_X,TIME_Y
231328,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.489538,0.999891
302995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.012670,0.611847
195984,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.157651,0.135587
88225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0.605984,0.011362
289842,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.108869,0.188526
82894,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.232153,0.077794
212207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.017742,0.632013
102925,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.168547,0.125649
196485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.121758,0.172994
290356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.097312,0.203618


In [20]:
len(list(le_ocg.classes_))

59

In [21]:
xValidation = crimes_time_validation.loc[:,'Aggravated Assault':'TIME_Y']
yValidation =  crimes_time_validation.loc[:,'DISTRICT_labels']
xValidation.drop([ 'MONTH', 'DAY_OF_MONTH', 'HOUR'], 1, inplace= True)
def create_model (type_layer= 'LSTM', n_layers=1, n_embeding = 32,n_epochs = 20,batch_size=512, top_k=3):
    
    layer_r = tf.keras.layers.LSTM(n_embeding, return_sequences=True)
    layer_e = tf.keras.layers.LSTM(n_embeding)
    if type_layer == 'GRU':
        layer_r = tf.keras.layers.GRU(n_embeding, return_sequences=True)
        layer_e = tf.keras.layers.GRU(n_embeding)
        
    model_i = tf.keras.models.Sequential()
    model_i.add(tf.keras.layers.Embedding(30, n_embeding, input_length=73))
    if n_layers != 0:
        for i in range(n_layers):
            model_i.add(layer_r)
    
    model_i.add(layer_e)
    
    model_i.add(tf.keras.layers.Dense(12, activation='softmax'))
    optimizer = tf.keras.optimizers.Adam(lr=0.01)
    metric= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
    model_i.compile (optimizer=optimizer,loss='sparse_categorical_crossentropy', metrics=[metric] )
    
    model_i.fit(xTrain, yTrain, epochs = n_epochs,batch_size=batch_size, validation_data=(xTest, yTest))
    return model_i.evaluate(xValidation, yValidation)


In [22]:
create_model (type_layer= 'LSTM', n_layers=0, n_embeding = 128,n_epochs = 20,batch_size=512 )

Epoch 1/20
357/357 [==============================] - 154s 430ms/step - loss: 2.3589 - sparse_top_k_categorical_accuracy: 0.4223 - val_loss: 2.3534 - val_sparse_top_k_categorical_accuracy: 0.4253
Epoch 2/20
357/357 [==============================] - 154s 432ms/step - loss: 2.3545 - sparse_top_k_categorical_accuracy: 0.4241 - val_loss: 2.3535 - val_sparse_top_k_categorical_accuracy: 0.4254
Epoch 3/20
357/357 [==============================] - 154s 433ms/step - loss: 2.3541 - sparse_top_k_categorical_accuracy: 0.4248 - val_loss: 2.3515 - val_sparse_top_k_categorical_accuracy: 0.4294
Epoch 4/20
357/357 [==============================] - 153s 428ms/step - loss: 2.3515 - sparse_top_k_categorical_accuracy: 0.4296 - val_loss: 2.3447 - val_sparse_top_k_categorical_accuracy: 0.4358
Epoch 5/20
357/357 [==============================] - 153s 428ms/step - loss: 2.3447 - sparse_top_k_categorical_accuracy: 0.4349 - val_loss: 2.3421 - val_sparse_top_k_categorical_accuracy: 0.4432
Epoch 6/20
357/357 [

[2.3472208976745605, 0.42494165897369385]

In [23]:

def grid_search(n_layers=[0,1,2,3], type_layers= ['GRU','LSTM'], n_embedings=[64,128], n_epochs = 20, top_ks=[3,4]):
    i=0
    lista = {}
    for n_layer in n_layers:
        for type_l in type_layers:
            for n_embeding in n_embedings:
                for top_k in top_ks:
                    print("************ NUEVA PRUEBA *************")
                    print (type_l)
                    print (n_layer)
                    print (n_embeding)
                    print (top_k)
                    try:
                        fit = create_model (type_layer= type_l, n_layers=n_layer, n_embeding = n_embeding, n_epochs=n_epochs)
                        print(fit)
                        print(i)
                        lista[str(i)] = "type_l=" + type_l + ",n_layer="+ str(n_layer) + ",n_embeding" + str(n_embeding) + ",top_k=" + str(top_k)
                        i=i+1
                    except:
                        print("Imposible entrenar el modelos con los parametros siguientes")
    return lista



In [24]:
resultados0 = grid_search (n_layers=[0],type_layers= ['GRU','LSTM'], n_embedings=[64,128], top_ks=[3,4])


************ NUEVA PRUEBA *************
GRU
0
64
3
Epoch 1/20
357/357 [==============================] - 45s 126ms/step - loss: 2.3571 - sparse_top_k_categorical_accuracy: 0.4231 - val_loss: 2.3531 - val_sparse_top_k_categorical_accuracy: 0.4262
Epoch 2/20
357/357 [==============================] - 44s 124ms/step - loss: 2.3544 - sparse_top_k_categorical_accuracy: 0.4252 - val_loss: 2.3537 - val_sparse_top_k_categorical_accuracy: 0.4263
Epoch 3/20
357/357 [==============================] - 44s 124ms/step - loss: 2.3531 - sparse_top_k_categorical_accuracy: 0.4275 - val_loss: 2.3484 - val_sparse_top_k_categorical_accuracy: 0.4346
Epoch 4/20
357/357 [==============================] - 45s 127ms/step - loss: 2.3568 - sparse_top_k_categorical_accuracy: 0.4266 - val_loss: 2.3634 - val_sparse_top_k_categorical_accuracy: 0.4081
Epoch 5/20
357/357 [==============================] - 45s 126ms/step - loss: 2.3594 - sparse_top_k_categorical_accuracy: 0.4234 - val_loss: 2.3599 - val_sparse_top_k_cat

[2.334188938140869, 0.44665220379829407]
1
************ NUEVA PRUEBA *************
GRU
0
128
3
Epoch 1/20
357/357 [==============================] - 114s 320ms/step - loss: 2.3576 - sparse_top_k_categorical_accuracy: 0.4222 - val_loss: 2.3537 - val_sparse_top_k_categorical_accuracy: 0.4264
Epoch 2/20
357/357 [==============================] - 114s 319ms/step - loss: 2.3549 - sparse_top_k_categorical_accuracy: 0.4252 - val_loss: 2.3533 - val_sparse_top_k_categorical_accuracy: 0.4298
Epoch 3/20
357/357 [==============================] - 114s 320ms/step - loss: 2.3537 - sparse_top_k_categorical_accuracy: 0.4273 - val_loss: 2.3535 - val_sparse_top_k_categorical_accuracy: 0.4352
Epoch 4/20
357/357 [==============================] - 113s 316ms/step - loss: 2.3604 - sparse_top_k_categorical_accuracy: 0.4215 - val_loss: 2.3556 - val_sparse_top_k_categorical_accuracy: 0.4266
Epoch 5/20
357/357 [==============================] - 119s 334ms/step - loss: 2.3563 - sparse_top_k_categorical_accuracy:

1902/1902 [==============================] - 16s 9ms/step - loss: 2.3609 - sparse_top_k_categorical_accuracy: 0.4155
[2.360860586166382, 0.41549158096313477]
3
************ NUEVA PRUEBA *************
LSTM
0
64
3
Epoch 1/20
357/357 [==============================] - 52s 147ms/step - loss: 2.3573 - sparse_top_k_categorical_accuracy: 0.4227 - val_loss: 2.3538 - val_sparse_top_k_categorical_accuracy: 0.4263
Epoch 2/20
357/357 [==============================] - 52s 146ms/step - loss: 2.3545 - sparse_top_k_categorical_accuracy: 0.4252 - val_loss: 2.3543 - val_sparse_top_k_categorical_accuracy: 0.4260
Epoch 3/20
357/357 [==============================] - 54s 150ms/step - loss: 2.3541 - sparse_top_k_categorical_accuracy: 0.4259 - val_loss: 2.3528 - val_sparse_top_k_categorical_accuracy: 0.4270
Epoch 4/20
357/357 [==============================] - 57s 159ms/step - loss: 2.3529 - sparse_top_k_categorical_accuracy: 0.4259 - val_loss: 2.3512 - val_sparse_top_k_categorical_accuracy: 0.4284
Epoch 5/

1902/1902 [==============================] - 12s 6ms/step - loss: 2.3263 - sparse_top_k_categorical_accuracy: 0.4476
[2.326326847076416, 0.4476054310798645]
5
************ NUEVA PRUEBA *************
LSTM
0
128
3
Epoch 1/20
357/357 [==============================] - 165s 462ms/step - loss: 2.3581 - sparse_top_k_categorical_accuracy: 0.4221 - val_loss: 2.3544 - val_sparse_top_k_categorical_accuracy: 0.4259
Epoch 2/20
357/357 [==============================] - 158s 444ms/step - loss: 2.3548 - sparse_top_k_categorical_accuracy: 0.4238 - val_loss: 2.3545 - val_sparse_top_k_categorical_accuracy: 0.4252
Epoch 3/20
357/357 [==============================] - 154s 432ms/step - loss: 2.3542 - sparse_top_k_categorical_accuracy: 0.4254 - val_loss: 2.3538 - val_sparse_top_k_categorical_accuracy: 0.4270
Epoch 4/20
357/357 [==============================] - 152s 426ms/step - loss: 2.3537 - sparse_top_k_categorical_accuracy: 0.4261 - val_loss: 2.3523 - val_sparse_top_k_categorical_accuracy: 0.4284
Epoc

1902/1902 [==============================] - 26s 14ms/step - loss: 2.3216 - sparse_top_k_categorical_accuracy: 0.4531
[2.3215808868408203, 0.4530946910381317]
7


In [25]:
resultados1 = grid_search (n_layers=[1],type_layers= ['LSTM'], n_embedings=[64,128], top_ks=[3,4])

************ NUEVA PRUEBA *************
LSTM
1
64
3
Epoch 1/20
357/357 [==============================] - 121s 339ms/step - loss: 2.3585 - sparse_top_k_categorical_accuracy: 0.4219 - val_loss: 2.3521 - val_sparse_top_k_categorical_accuracy: 0.4276
Epoch 2/20
357/357 [==============================] - 124s 346ms/step - loss: 2.3514 - sparse_top_k_categorical_accuracy: 0.4308 - val_loss: 2.3488 - val_sparse_top_k_categorical_accuracy: 0.4347
Epoch 3/20
357/357 [==============================] - 124s 348ms/step - loss: 2.3449 - sparse_top_k_categorical_accuracy: 0.4346 - val_loss: 2.3400 - val_sparse_top_k_categorical_accuracy: 0.4367
Epoch 4/20
357/357 [==============================] - 124s 347ms/step - loss: 2.3509 - sparse_top_k_categorical_accuracy: 0.4283 - val_loss: 2.3530 - val_sparse_top_k_categorical_accuracy: 0.4258
Epoch 5/20
357/357 [==============================] - 124s 347ms/step - loss: 2.3507 - sparse_top_k_categorical_accuracy: 0.4294 - val_loss: 2.3430 - val_sparse_top

1902/1902 [==============================] - 27s 14ms/step - loss: 2.3269 - sparse_top_k_categorical_accuracy: 0.4438
[2.3268938064575195, 0.44377610087394714]
1
************ NUEVA PRUEBA *************
LSTM
1
128
3
Epoch 1/20
357/357 [==============================] - 334s 935ms/step - loss: 2.3611 - sparse_top_k_categorical_accuracy: 0.4215 - val_loss: 2.3512 - val_sparse_top_k_categorical_accuracy: 0.4315
Epoch 2/20
357/357 [==============================] - 349s 977ms/step - loss: 2.3558 - sparse_top_k_categorical_accuracy: 0.4243 - val_loss: 2.3544 - val_sparse_top_k_categorical_accuracy: 0.4258
Epoch 3/20
357/357 [==============================] - 376s 1s/step - loss: 2.3554 - sparse_top_k_categorical_accuracy: 0.4244 - val_loss: 2.3545 - val_sparse_top_k_categorical_accuracy: 0.4258
Epoch 4/20
357/357 [==============================] - 369s 1s/step - loss: 2.3543 - sparse_top_k_categorical_accuracy: 0.4242 - val_loss: 2.3528 - val_sparse_top_k_categorical_accuracy: 0.4268
Epoch 5

In [28]:
n_embeding=20

model_wo_D = tf.keras.models.Sequential()
model_wo_D.add(tf.keras.layers.Dense(500, activation='relu', input_dim=73))
model_wo_D.add(tf.keras.layers.Dense(250, activation='relu'))
model_wo_D.add(tf.keras.layers.Dense(100, activation='relu'))
model_wo_D.add(tf.keras.layers.Dense(50, activation='relu'))
model_wo_D.add(tf.keras.layers.Dense(12, activation='softmax'))
metric_wo_D= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
model_wo_D.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=[metric_wo_D] )
model_fitted_wo_D = model_wo_D.fit(xTrain, yTrain, epochs = 100,batch_size=128, validation_data=(xTest, yTest))
results_wo_D = model_wo_D.evaluate(xValidation, yValidation)

Epoch 1/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3071 - sparse_top_k_categorical_accuracy: 0.4702 - val_loss: 2.2987 - val_sparse_top_k_categorical_accuracy: 0.4738
Epoch 2/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2933 - sparse_top_k_categorical_accuracy: 0.4773 - val_loss: 2.2948 - val_sparse_top_k_categorical_accuracy: 0.4762
Epoch 3/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2899 - sparse_top_k_categorical_accuracy: 0.4778 - val_loss: 2.2934 - val_sparse_top_k_categorical_accuracy: 0.4768
Epoch 4/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2867 - sparse_top_k_categorical_accuracy: 0.4802 - val_loss: 2.2926 - val_sparse_top_k_categorical_accuracy: 0.4792
Epoch 5/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2840 - sparse_top_k_categorical_accuracy: 0.4809 - val_loss: 2.2947 - val_sparse_top_k_categorical_accuracy: 0.4786
Epoch 6/100
1427/1427 [==

Epoch 43/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1890 - sparse_top_k_categorical_accuracy: 0.5180 - val_loss: 2.3807 - val_sparse_top_k_categorical_accuracy: 0.4567
Epoch 44/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1875 - sparse_top_k_categorical_accuracy: 0.5189 - val_loss: 2.3764 - val_sparse_top_k_categorical_accuracy: 0.4576
Epoch 45/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1856 - sparse_top_k_categorical_accuracy: 0.5203 - val_loss: 2.3774 - val_sparse_top_k_categorical_accuracy: 0.4584
Epoch 46/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1842 - sparse_top_k_categorical_accuracy: 0.5200 - val_loss: 2.3801 - val_sparse_top_k_categorical_accuracy: 0.4589
Epoch 47/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1826 - sparse_top_k_categorical_accuracy: 0.5214 - val_loss: 2.3824 - val_sparse_top_k_categorical_accuracy: 0.4595
Epoch 48/100
1427/14

Epoch 85/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1360 - sparse_top_k_categorical_accuracy: 0.5375 - val_loss: 2.4731 - val_sparse_top_k_categorical_accuracy: 0.4529
Epoch 86/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1362 - sparse_top_k_categorical_accuracy: 0.5373 - val_loss: 2.4720 - val_sparse_top_k_categorical_accuracy: 0.4500
Epoch 87/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1349 - sparse_top_k_categorical_accuracy: 0.5376 - val_loss: 2.4788 - val_sparse_top_k_categorical_accuracy: 0.4521
Epoch 88/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1334 - sparse_top_k_categorical_accuracy: 0.5384 - val_loss: 2.4838 - val_sparse_top_k_categorical_accuracy: 0.4526
Epoch 89/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1324 - sparse_top_k_categorical_accuracy: 0.5388 - val_loss: 2.4844 - val_sparse_top_k_categorical_accuracy: 0.4480
Epoch 90/100
1427/14

In [34]:
print(results_wo_D)

[2.5692942142486572, 0.4270288944244385]


In [29]:
n_embeding=20

model_wD = tf.keras.models.Sequential()
model_wD.add(tf.keras.layers.Dense(500, activation='relu', input_dim=73))
model_wD.add(tf.keras.layers.Dense(250, activation='relu'))
model_wD.add(tf.keras.layers.Dropout(0.25))
model_wD.add(tf.keras.layers.Dense(100, activation='relu'))
model_wD.add(tf.keras.layers.Dense(50, activation='relu'))
model_wD.add(tf.keras.layers.Dense(12, activation='softmax'))
metric_wD= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
model_wD.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=[metric_wD] )
model_fitted_wD = model_wD.fit(xTrain, yTrain, epochs = 100,batch_size=128, validation_data=(xTest, yTest))
results_wD = model_wD.evaluate(xValidation, yValidation)

Epoch 1/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3115 - sparse_top_k_categorical_accuracy: 0.4681 - val_loss: 2.2979 - val_sparse_top_k_categorical_accuracy: 0.4749
Epoch 2/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2960 - sparse_top_k_categorical_accuracy: 0.4759 - val_loss: 2.2944 - val_sparse_top_k_categorical_accuracy: 0.4783
Epoch 3/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2919 - sparse_top_k_categorical_accuracy: 0.4774 - val_loss: 2.2945 - val_sparse_top_k_categorical_accuracy: 0.4747
Epoch 4/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2895 - sparse_top_k_categorical_accuracy: 0.4782 - val_loss: 2.2919 - val_sparse_top_k_categorical_accuracy: 0.4776
Epoch 5/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2869 - sparse_top_k_categorical_accuracy: 0.4800 - val_loss: 2.2924 - val_sparse_top_k_categorical_accuracy: 0.4781
Epoch 6/100
1427/1427 [==

Epoch 43/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2279 - sparse_top_k_categorical_accuracy: 0.5032 - val_loss: 2.3293 - val_sparse_top_k_categorical_accuracy: 0.4666
Epoch 44/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2264 - sparse_top_k_categorical_accuracy: 0.5045 - val_loss: 2.3298 - val_sparse_top_k_categorical_accuracy: 0.4690
Epoch 45/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2257 - sparse_top_k_categorical_accuracy: 0.5051 - val_loss: 2.3326 - val_sparse_top_k_categorical_accuracy: 0.4689
Epoch 46/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2246 - sparse_top_k_categorical_accuracy: 0.5056 - val_loss: 2.3298 - val_sparse_top_k_categorical_accuracy: 0.4642
Epoch 47/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2237 - sparse_top_k_categorical_accuracy: 0.5055 - val_loss: 2.3366 - val_sparse_top_k_categorical_accuracy: 0.4677
Epoch 48/100
1427/14

Epoch 85/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1983 - sparse_top_k_categorical_accuracy: 0.5152 - val_loss: 2.3671 - val_sparse_top_k_categorical_accuracy: 0.4631
Epoch 86/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1978 - sparse_top_k_categorical_accuracy: 0.5153 - val_loss: 2.3681 - val_sparse_top_k_categorical_accuracy: 0.4618
Epoch 87/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1961 - sparse_top_k_categorical_accuracy: 0.5149 - val_loss: 2.3660 - val_sparse_top_k_categorical_accuracy: 0.4626
Epoch 88/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1960 - sparse_top_k_categorical_accuracy: 0.5154 - val_loss: 2.3745 - val_sparse_top_k_categorical_accuracy: 0.4621
Epoch 89/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1965 - sparse_top_k_categorical_accuracy: 0.5157 - val_loss: 2.3675 - val_sparse_top_k_categorical_accuracy: 0.4640
Epoch 90/100
1427/14

In [30]:
n_embeding=20

model_wDD = tf.keras.models.Sequential()
model_wDD.add(tf.keras.layers.Dense(512, activation='relu', input_dim=73))
model_wDD.add(tf.keras.layers.Dense(256, activation='relu'))
model_wDD.add(tf.keras.layers.Dropout(0.25))
model_wDD.add(tf.keras.layers.Dense(128, activation='relu'))
model_wDD.add(tf.keras.layers.Dense(128, activation='relu'))
model_wDD.add(tf.keras.layers.Dropout(0.25))
model_wDD.add(tf.keras.layers.Dense(64, activation='relu'))
model_wDD.add(tf.keras.layers.Dense(12, activation='softmax'))
metric_wDD= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
model_wDD.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=[metric_wDD] )
model_fitted_wDD = model_wDD.fit(xTrain, yTrain, epochs = 100,batch_size=128, validation_data=(xTest, yTest))
results_wDD = model_wDD.evaluate(xValidation, yValidation)

Epoch 1/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.3156 - sparse_top_k_categorical_accuracy: 0.4654 - val_loss: 2.2997 - val_sparse_top_k_categorical_accuracy: 0.4758
Epoch 2/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.3005 - sparse_top_k_categorical_accuracy: 0.4744 - val_loss: 2.2943 - val_sparse_top_k_categorical_accuracy: 0.4750
Epoch 3/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2969 - sparse_top_k_categorical_accuracy: 0.4757 - val_loss: 2.2943 - val_sparse_top_k_categorical_accuracy: 0.4777
Epoch 4/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2937 - sparse_top_k_categorical_accuracy: 0.4768 - val_loss: 2.2938 - val_sparse_top_k_categorical_accuracy: 0.4781
Epoch 5/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2918 - sparse_top_k_categorical_accuracy: 0.4780 - val_loss: 2.2924 - val_sparse_top_k_categorical_accuracy: 0.4794
Epoch 6/100
1427/1427 [==

Epoch 43/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2536 - sparse_top_k_categorical_accuracy: 0.4946 - val_loss: 2.3097 - val_sparse_top_k_categorical_accuracy: 0.4728
Epoch 44/100
1427/1427 [==============================] - 8s 5ms/step - loss: 2.2532 - sparse_top_k_categorical_accuracy: 0.4954 - val_loss: 2.3093 - val_sparse_top_k_categorical_accuracy: 0.4725
Epoch 45/100
1427/1427 [==============================] - 8s 5ms/step - loss: 2.2536 - sparse_top_k_categorical_accuracy: 0.4955 - val_loss: 2.3098 - val_sparse_top_k_categorical_accuracy: 0.4718
Epoch 46/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2511 - sparse_top_k_categorical_accuracy: 0.4959 - val_loss: 2.3111 - val_sparse_top_k_categorical_accuracy: 0.4718
Epoch 47/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2514 - sparse_top_k_categorical_accuracy: 0.4951 - val_loss: 2.3113 - val_sparse_top_k_categorical_accuracy: 0.4709
Epoch 48/100
1427/14

Epoch 85/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2321 - sparse_top_k_categorical_accuracy: 0.5037 - val_loss: 2.3290 - val_sparse_top_k_categorical_accuracy: 0.4685
Epoch 86/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2311 - sparse_top_k_categorical_accuracy: 0.5040 - val_loss: 2.3388 - val_sparse_top_k_categorical_accuracy: 0.4691
Epoch 87/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2306 - sparse_top_k_categorical_accuracy: 0.5047 - val_loss: 2.3342 - val_sparse_top_k_categorical_accuracy: 0.4681
Epoch 88/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2297 - sparse_top_k_categorical_accuracy: 0.5044 - val_loss: 2.3292 - val_sparse_top_k_categorical_accuracy: 0.4674
Epoch 89/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2308 - sparse_top_k_categorical_accuracy: 0.5042 - val_loss: 2.3321 - val_sparse_top_k_categorical_accuracy: 0.4685
Epoch 90/100
1427/14

In [35]:
print(results_wDD)

[2.3696868419647217, 0.4494297206401825]


In [31]:
n_embeding=20

model_wo_D4 = tf.keras.models.Sequential()
model_wo_D4.add(tf.keras.layers.Dense(500, activation='relu', input_dim=73))
model_wo_D4.add(tf.keras.layers.Dense(250, activation='relu'))
model_wo_D4.add(tf.keras.layers.Dense(100, activation='relu'))
model_wo_D4.add(tf.keras.layers.Dense(50, activation='relu'))
model_wo_D4.add(tf.keras.layers.Dense(12, activation='softmax'))
metric_wo_D4= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
model_wo_D4.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=[metric_wo_D4] )
model_fitted_wo_D4 = model_wo_D4.fit(xTrain, yTrain, epochs = 100,batch_size=128, validation_data=(xTest, yTest))
results_wo_D4 = model_wo_D4.evaluate(xValidation, yValidation)

Epoch 1/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.3085 - sparse_top_k_categorical_accuracy: 0.4688 - val_loss: 2.2971 - val_sparse_top_k_categorical_accuracy: 0.4771
Epoch 2/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.2940 - sparse_top_k_categorical_accuracy: 0.4776 - val_loss: 2.2959 - val_sparse_top_k_categorical_accuracy: 0.4772
Epoch 3/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2899 - sparse_top_k_categorical_accuracy: 0.4790 - val_loss: 2.2943 - val_sparse_top_k_categorical_accuracy: 0.4786
Epoch 4/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2868 - sparse_top_k_categorical_accuracy: 0.4794 - val_loss: 2.2926 - val_sparse_top_k_categorical_accuracy: 0.4783
Epoch 5/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2843 - sparse_top_k_categorical_accuracy: 0.4812 - val_loss: 2.2925 - val_sparse_top_k_categorical_accuracy: 0.4779
Epoch 6/100
1427/1427 [==

Epoch 43/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.1911 - sparse_top_k_categorical_accuracy: 0.5185 - val_loss: 2.3751 - val_sparse_top_k_categorical_accuracy: 0.4619
Epoch 44/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.1892 - sparse_top_k_categorical_accuracy: 0.5198 - val_loss: 2.3794 - val_sparse_top_k_categorical_accuracy: 0.4551
Epoch 45/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.1875 - sparse_top_k_categorical_accuracy: 0.5206 - val_loss: 2.3791 - val_sparse_top_k_categorical_accuracy: 0.4558
Epoch 46/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.1861 - sparse_top_k_categorical_accuracy: 0.5208 - val_loss: 2.3849 - val_sparse_top_k_categorical_accuracy: 0.4593
Epoch 47/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.1843 - sparse_top_k_categorical_accuracy: 0.5211 - val_loss: 2.3771 - val_sparse_top_k_categorical_accuracy: 0.4579
Epoch 48/100
1427/14

Epoch 85/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.1398 - sparse_top_k_categorical_accuracy: 0.5370 - val_loss: 2.4650 - val_sparse_top_k_categorical_accuracy: 0.4465
Epoch 86/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1381 - sparse_top_k_categorical_accuracy: 0.5368 - val_loss: 2.4634 - val_sparse_top_k_categorical_accuracy: 0.4497
Epoch 87/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.1376 - sparse_top_k_categorical_accuracy: 0.5373 - val_loss: 2.4674 - val_sparse_top_k_categorical_accuracy: 0.4505
Epoch 88/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.1364 - sparse_top_k_categorical_accuracy: 0.5381 - val_loss: 2.4716 - val_sparse_top_k_categorical_accuracy: 0.4511
Epoch 89/100
1427/1427 [==============================] - 5s 4ms/step - loss: 2.1353 - sparse_top_k_categorical_accuracy: 0.5381 - val_loss: 2.4717 - val_sparse_top_k_categorical_accuracy: 0.4492
Epoch 90/100
1427/14

In [32]:
n_embeding=20

model_wD4 = tf.keras.models.Sequential()
model_wD4.add(tf.keras.layers.Dense(500, activation='relu', input_dim=73))
model_wD4.add(tf.keras.layers.Dense(250, activation='relu'))
model_wD4.add(tf.keras.layers.Dropout(0.25))
model_wD4.add(tf.keras.layers.Dense(100, activation='relu'))
model_wD4.add(tf.keras.layers.Dense(50, activation='relu'))
model_wD4.add(tf.keras.layers.Dense(12, activation='softmax'))
metric_wD4= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
model_wD4.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=[metric_wD4] )
model_fitted_wD4 = model_wD4.fit(xTrain, yTrain, epochs = 100,batch_size=128, validation_data=(xTest, yTest))
results_wD4 = model_wD4.evaluate(xValidation, yValidation)

Epoch 1/100
1427/1427 [==============================] - 6s 5ms/step - loss: 2.3100 - sparse_top_k_categorical_accuracy: 0.4680 - val_loss: 2.2985 - val_sparse_top_k_categorical_accuracy: 0.4765
Epoch 2/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2957 - sparse_top_k_categorical_accuracy: 0.4771 - val_loss: 2.2959 - val_sparse_top_k_categorical_accuracy: 0.4754
Epoch 3/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2919 - sparse_top_k_categorical_accuracy: 0.4773 - val_loss: 2.2938 - val_sparse_top_k_categorical_accuracy: 0.4781
Epoch 4/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2891 - sparse_top_k_categorical_accuracy: 0.4789 - val_loss: 2.2918 - val_sparse_top_k_categorical_accuracy: 0.4782
Epoch 5/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2870 - sparse_top_k_categorical_accuracy: 0.4809 - val_loss: 2.2916 - val_sparse_top_k_categorical_accuracy: 0.4784
Epoch 6/100
1427/1427 [==

Epoch 43/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2263 - sparse_top_k_categorical_accuracy: 0.5050 - val_loss: 2.3296 - val_sparse_top_k_categorical_accuracy: 0.4654
Epoch 44/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2255 - sparse_top_k_categorical_accuracy: 0.5046 - val_loss: 2.3376 - val_sparse_top_k_categorical_accuracy: 0.4649
Epoch 45/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2241 - sparse_top_k_categorical_accuracy: 0.5073 - val_loss: 2.3322 - val_sparse_top_k_categorical_accuracy: 0.4676
Epoch 46/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2238 - sparse_top_k_categorical_accuracy: 0.5060 - val_loss: 2.3340 - val_sparse_top_k_categorical_accuracy: 0.4673
Epoch 47/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.2227 - sparse_top_k_categorical_accuracy: 0.5061 - val_loss: 2.3355 - val_sparse_top_k_categorical_accuracy: 0.4639
Epoch 48/100
1427/14

Epoch 85/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1972 - sparse_top_k_categorical_accuracy: 0.5157 - val_loss: 2.3631 - val_sparse_top_k_categorical_accuracy: 0.4619
Epoch 86/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1951 - sparse_top_k_categorical_accuracy: 0.5174 - val_loss: 2.3706 - val_sparse_top_k_categorical_accuracy: 0.4598
Epoch 87/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1957 - sparse_top_k_categorical_accuracy: 0.5159 - val_loss: 2.3714 - val_sparse_top_k_categorical_accuracy: 0.4576
Epoch 88/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1943 - sparse_top_k_categorical_accuracy: 0.5172 - val_loss: 2.3702 - val_sparse_top_k_categorical_accuracy: 0.4614
Epoch 89/100
1427/1427 [==============================] - 6s 4ms/step - loss: 2.1939 - sparse_top_k_categorical_accuracy: 0.5181 - val_loss: 2.3716 - val_sparse_top_k_categorical_accuracy: 0.4622
Epoch 90/100
1427/14

In [33]:
n_embeding=20

model_wDD4 = tf.keras.models.Sequential()
model_wDD4.add(tf.keras.layers.Dense(512, activation='relu', input_dim=73))
model_wDD4.add(tf.keras.layers.Dense(256, activation='relu'))
model_wDD4.add(tf.keras.layers.Dropout(0.25))
model_wDD4.add(tf.keras.layers.Dense(128, activation='relu'))
model_wDD4.add(tf.keras.layers.Dense(128, activation='relu'))
model_wDD4.add(tf.keras.layers.Dropout(0.25))
model_wDD4.add(tf.keras.layers.Dense(64, activation='relu'))
model_wDD4.add(tf.keras.layers.Dense(12, activation='softmax'))
metric_wDD4= tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="sparse_top_k_categorical_accuracy", dtype=None)
model_wDD4.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=[metric_wDD4] )
model_fitted_wDD4= model_wDD4.fit(xTrain, yTrain, epochs = 100,batch_size=128, validation_data=(xTest, yTest))
results_wDD4 = model_wDD4.evaluate(xValidation, yValidation)

Epoch 1/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.3156 - sparse_top_k_categorical_accuracy: 0.4654 - val_loss: 2.3005 - val_sparse_top_k_categorical_accuracy: 0.4726
Epoch 2/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.3008 - sparse_top_k_categorical_accuracy: 0.4736 - val_loss: 2.2969 - val_sparse_top_k_categorical_accuracy: 0.4772
Epoch 3/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2970 - sparse_top_k_categorical_accuracy: 0.4750 - val_loss: 2.2934 - val_sparse_top_k_categorical_accuracy: 0.4750
Epoch 4/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2945 - sparse_top_k_categorical_accuracy: 0.4762 - val_loss: 2.2930 - val_sparse_top_k_categorical_accuracy: 0.4782
Epoch 5/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2925 - sparse_top_k_categorical_accuracy: 0.4771 - val_loss: 2.2924 - val_sparse_top_k_categorical_accuracy: 0.4754
Epoch 6/100
1427/1427 [==

Epoch 43/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2558 - sparse_top_k_categorical_accuracy: 0.4939 - val_loss: 2.3063 - val_sparse_top_k_categorical_accuracy: 0.4723
Epoch 44/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2551 - sparse_top_k_categorical_accuracy: 0.4930 - val_loss: 2.3077 - val_sparse_top_k_categorical_accuracy: 0.4723
Epoch 45/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2539 - sparse_top_k_categorical_accuracy: 0.4950 - val_loss: 2.3101 - val_sparse_top_k_categorical_accuracy: 0.4721
Epoch 46/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2536 - sparse_top_k_categorical_accuracy: 0.4952 - val_loss: 2.3095 - val_sparse_top_k_categorical_accuracy: 0.4727
Epoch 47/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2531 - sparse_top_k_categorical_accuracy: 0.4950 - val_loss: 2.3113 - val_sparse_top_k_categorical_accuracy: 0.4711
Epoch 48/100
1427/14

Epoch 85/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2344 - sparse_top_k_categorical_accuracy: 0.5022 - val_loss: 2.3292 - val_sparse_top_k_categorical_accuracy: 0.4698
Epoch 86/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2343 - sparse_top_k_categorical_accuracy: 0.5020 - val_loss: 2.3267 - val_sparse_top_k_categorical_accuracy: 0.4700
Epoch 87/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2334 - sparse_top_k_categorical_accuracy: 0.5016 - val_loss: 2.3308 - val_sparse_top_k_categorical_accuracy: 0.4688
Epoch 88/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2325 - sparse_top_k_categorical_accuracy: 0.5033 - val_loss: 2.3331 - val_sparse_top_k_categorical_accuracy: 0.4656
Epoch 89/100
1427/1427 [==============================] - 7s 5ms/step - loss: 2.2329 - sparse_top_k_categorical_accuracy: 0.5035 - val_loss: 2.3318 - val_sparse_top_k_categorical_accuracy: 0.4659
Epoch 90/100
1427/14

In [55]:
print(len(xValidation))
mapping = dict(zip(le_d.classes_, range(len(le_d.classes_))))
print(mapping)
predict = model_wDD.predict_classes(xValidation)
unique, counts = np.unique(predict, return_counts=True)
print (np.asarray((unique, counts)).T)
unique, counts = np.unique(yValidation, return_counts=True)

print (np.asarray((unique, counts)).T)

60846
{'A1': 0, 'A15': 1, 'A7': 2, 'B2': 3, 'B3': 4, 'C11': 5, 'C6': 6, 'D14': 7, 'D4': 8, 'E13': 9, 'E18': 10, 'E5': 11}
[[    0  5622]
 [    1   101]
 [    2   115]
 [    3 22831]
 [    4  3933]
 [    5 12201]
 [    6   538]
 [    7   982]
 [    8 14177]
 [    9   119]
 [   10   103]
 [   11   124]]
[[   0 6637]
 [   1 1199]
 [   2 2446]
 [   3 9265]
 [   4 6963]
 [   5 8124]
 [   6 4831]
 [   7 3846]
 [   8 7939]
 [   9 3412]
 [  10 3583]
 [  11 2601]]


In [56]:
print(len(xValidation))
mapping = dict(zip(le_d.classes_, range(len(le_d.classes_))))
print(mapping)
predict = model_wD.predict_classes(xValidation)
unique, counts = np.unique(predict, return_counts=True)
print (np.asarray((unique, counts)).T)
unique, counts = np.unique(yValidation, return_counts=True)

print (np.asarray((unique, counts)).T)

60846
{'A1': 0, 'A15': 1, 'A7': 2, 'B2': 3, 'B3': 4, 'C11': 5, 'C6': 6, 'D14': 7, 'D4': 8, 'E13': 9, 'E18': 10, 'E5': 11}
[[    0  6267]
 [    1    96]
 [    2   192]
 [    3 19531]
 [    4  5422]
 [    5 14182]
 [    6   755]
 [    7  1144]
 [    8 12484]
 [    9   244]
 [   10   317]
 [   11   212]]
[[   0 6637]
 [   1 1199]
 [   2 2446]
 [   3 9265]
 [   4 6963]
 [   5 8124]
 [   6 4831]
 [   7 3846]
 [   8 7939]
 [   9 3412]
 [  10 3583]
 [  11 2601]]
